In [22]:
%pip install -Uq llama-index-llms-groq
%pip install -Uq llama-index
%pip install -Uq python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# llm v0

In [ ]:
import os
from llama_index.core.llms import ChatMessage
from llama_index.llms.groq import Groq
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

SYSTEM_PROMPT = '''

**Objective**:
Transform web-scraped markdown documents into a clear, comprehensive representation of essential content. The focus is on extracting all valuable information, ensuring that the main content of the page is fully captured and presented. While filtering out repetitive and irrelevant text, ensure that the content length is preserved and formatted into a well-structured markdown format. Ensure the output reflects the true content, context, and structure of the original webpage, maintaining the same language as the input document.

---

**Instructions for the System**:

1. **Initial Header**:  
   - Begin each transformed output with: `"# Response for <url>"`, inserting the URL found in the file. This acts as a consistent identifier for the source of the content.

2. **Complete Core Content Identification and Extraction**:  
   - Ensure that all main content from the raw markdown file is thoroughly examined and extracted, maintaining the full content length.
   - This includes detailed articles, main text bodies, descriptions, and any substantial information.
   - Guarantee that all content of informational value is included; nothing should be omitted that contributes to the page's main message and context.

3. **Link Evaluation and Inclusion**:
   - **Preserve** links only if they provide significant relevance, additional details, or direct relevance to the core content.
   - **Exclude** links commonly found in navigation menus, headers, footers, or site-wide elements like terms of service and privacy policy unless directly relevant to the page content.

4. **Information Structure and Formatting**:
   - Employ markdown syntax to organize the output effectively, using headings, subheadings, lists, and bullet points to replicate the hierarchical and categorical structure of the original content.
   - Ensure that content flows logically and contextually, maintaining the essence and message of the source material.
   - Highlight and retain any contextually important links using markdown `[link text](URL)` format.

5. **Content Integrity and Verbosity**:
   - Keep the extracted content faithful to its original form without paraphrasing, ensuring a detailed representation that captures all essential information and nuances.
   - Deliver a verbose output that preserves the full length of the original content, presenting an accurate and immediately usable document.

6. **Quality and Consistency**:
   - Ensure precision and uniformity across outputs for different input files, respecting the specificity and uniqueness of each webpage’s content.
   - Pay strict attention to the details that characterize and define the content’s context and purpose.

7. **Language Consistency**:
   - Always produce the output in the same language as the raw input document to maintain linguistic accuracy and coherence.

---

**Content Processing Instructions**:

- **Pattern-based Noise Removal**:
  - Remove elements that are typically navigational or promotional, such as:
    - Menu items (e.g., "Cursos", "Notícias", "Investigação", "Eventos", "Candidaturas").
    - Links unrelated to the main topic like "Página Inicial", "Office 365", or "Moodle".
    - Social media links (e.g., "Facebook", "Twitter", "Instagram").
    - Administrative or utility links and text items (e.g., "Login", "Subscribe", "Newsletter").
    - Repeated elements like footers and standard disclaimers.

- **Focus Extraction**:
  - Concentrate on sections that exhibit clearly defined information, such as structured headings in the raw markdown that correlate with substantive text.

- **Link Relevance Check**:
  - Retain links that significantly enhance the content, disregarding generic site links unless directly tied to the primary content focus.

- **Content Authenticity**:
  - Preserve the authenticity and factual representation of the webpage’s information, ensuring it aligns with the original page's context and message.

---

**Reminder**: This prompt is a guideline designed to cover a wide variety of content while consistently achieving the objective of extracting and structuring meaningful information. Always adapt according to the unique characteristics presented within each file, ensuring completeness, full capture of the main content with its length preserved, and maintaining the original language.

'''

APPEND_PROMPT = "RESPIRA FUNDO, REVÊ CUIDADOSAMENTE O CONTEÚDO ORIGINAL, E AGORA COMPLETA A VERSÃO FINAL DO CONTEÚDO FORMATADO. POR FAVOR, INCLUI TODA A INFORMAÇÃO RELEVANTE E FILTRA QUALQUER RUÍDO."

REVISOR_PROMPT_bk_0 = '''

Por favor, revê atentamente o teu texto formatado tendo em conta as seguintes instruções. Analisa o resultado anterior gerado pelo modelo juntamente com o prompt do sistema e o conteúdo bruto original. O objetivo é assegurar que o documento final formatado atende a todos os requisitos do sistema, capturando toda a informação relevante e mantendo a integridade do conteúdo.

1. **Compreensão Completa**: Verifica se todo o conteúdo principal do documento original foi capturado. Assegura que nenhuma informação relevante foi omitida e que o texto reflete fielmente o conteúdo original.

2. **Estrutura e Formatação**: Confirma que o texto está devidamente estruturado em markdown, utilizando cabeçalhos, subtítulos e listas para organizar a informação de forma lógica e categórica.

3. **Coerência de Língua**: Garante que o output é apresentado na mesma língua que o documento original, preservando a coerência linguística.

4. **Links e Contexto**: Certifica-te de que todos os links importantes e contextualmente relevantes foram incluídos, enquanto os links de navegação e não relevantes foram eliminados.

5. **Preservação do Conteúdo**: Assegura-te de que o comprimento do conteúdo não foi reduzido, mantendo toda a informação essencial e nuances do texto original.

Realiza as correções, adições ou ajustes necessários para alinhar o documento final com os requisitos do sistema. Quando terminares, o output deverá ser uma representação fiel, completa e bem estruturada do conteúdo original, pronto para utilização imediata sem necessidade de edição adicional.

'''


REVISOR_PROMPT = '''

Por favor, revê atentamente o teu texto formatado tendo em conta as seguintes instruções. Analisa o resultado anterior gerado pelo modelo juntamente com o SYSTEM PROMPT e o conteúdo bruto original. O objetivo é assegurar que o documento final formatado atende a todos os requisitos do sistema, capturando toda a informação relevante e mantendo a integridade do conteúdo.

1. **Compreensão Completa**: Verifica se todo o conteúdo principal do documento original foi capturado. Assegura que nenhuma informação relevante foi omitida e que o texto reflete fielmente o conteúdo original.

2. **Estrutura e Formatação**: Confirma que o texto está devidamente estruturado em markdown, utilizando cabeçalhos, subtítulos e listas para organizar a informação de forma lógica e categórica.

3. **Coerência de Língua**: Garante que o output é apresentado na mesma língua que o documento original, preservando a coerência linguística.

4. **Links e Contexto**: Certifica-te de que todos os links importantes e contextualmente relevantes foram incluídos, enquanto os links de navegação e não relevantes foram eliminados.

5. **Preservação do Conteúdo**: Assegura-te de que o comprimento do conteúdo não foi reduzido, mantendo toda a informação essencial e nuances do texto original.

Realiza as correções, adições ou ajustes necessários para alinhar o documento final com os requisitos do sistema. Quando terminares, o output deverá ser uma representação fiel, completa e bem estruturada do conteúdo original, pronto para utilização imediata sem necessidade de edição adicional.

**Nota super importante**: Ao responder, o modelo nunca deve incluir frases que são típicas de textos gerados por LLM que visam soar formais e estruturados, mas que muitas vezes parecem genéricas ou desnecessárias. Para evitar isso, é essencial usar uma linguagem direta e concisa, eliminando palavras ou frases de preenchimento que não acrescentam valor. O modelo deve priorizar a clareza e a concisão, apresentando apenas a informação essencial ou o resultado específico de forma direta. 

### O que descartar:

Evita incluir exemplos como a listagem abaixo, que contém informações de navegação, links de serviços e outras referências irrelevantes para o propósito da análise:

```markdown
## Universidade Lusófona
### Apresentação

A Universidade Lusófona é uma instituição de ensino superior que oferece uma variedade de cursos e programas acadêmicos.

### Serviços

- **Contacts**: https://www.ulusofona.pt/en/contacts
- **Password Change and Recovery**: https://secure.ensinolusofona.pt/alteracao_password/f?p=133:2
- **Help us to improve**: https://ulusofona.typeform.com/to/cipp2UFI
- **Lost and Found**: https://www.ulusofona.pt/en/lost-and-found

### Cursos

- **Bachelors**: https://www.ulusofona.pt/en/undergraduate
- **Masters**: https://www.ulusofona.pt/en/masters
- **PhD**: https://www.ulusofona.pt/en/phd
- **Post-graduation**: https://www.ulusofona.pt/en/post-graduation
- **All the courses**: https://www.ulusofona.pt/en/courses

### Documentos

- **Fees and Emoluments**: https://www.ulusofona.pt/en/documents?cat=5
- **Regulations and Orders**: https://www.ulusofona.pt/en/documents?cat=1
- **Forms**: https://www.ulusofona.pt/en/documents?cat=13
- **Reports**: https://www.ulusofona.pt/en/documents?cat=4
- **Validation of documents**: https://www.ulusofona.pt/en/validation-of-issued-documents

### Informação Acadêmica

- **Calendários Acadêmicos**: https://www.ulusofona.pt/en/calendars
- **Faculdades e Escolas**: https://www.ulusofona.pt/en/faculties-and-schools
- **Plano de Género e Diversidade**: https://www.ensinolusofona.pt/en/gender-and-diversity-plan
- **Propinas**: https://www.ulusofona.pt/en/fees
- **Motivos para Frequência**: https://razoes.ulusofona.pt/
- **Qualidade**: https://www.ulusofona.pt/en/qualidade
- **Sobre Nós**: https://www.ulusofona.pt/en/about
- **Visite-nos**: https://ulusofona.typeform.com/to/ypj6qk
- **Canal de Denúncias Internas**: https://www.ulusofona.pt/en/internal-reporting-channel

### Infraestruturas

- **Campus**: https://campus.ulusofona.pt/
- **Contactos**: https://www.ulusofona.pt/en/contacts
- **Entidade Promotora**: https://www.cofac.pt
- **Lusófona no Mundo**: https://www.ensinolusofona.pt/en/
- **Lusófona 360º**: https://vr360.ulusofona.pt/visitavirtual_EN/

### Professores

- **Avadoc**: https://www.ulusofona.pt/avadoc
- **Bem-vindos**: https://boasvindas.ulusofona.pt/
- **Carreira Docente**: https://www.ulusofona.pt/en/documents?q=Career
- **Diretório**: https://diretorio.ulusofona.pt/
- **Emprego Científico**: https://www.ulusofona.pt/en/open-positions/scientific-employment-and-research-grants
- **Portal do Docente**: https://secure.ensinolusofona.pt/ficha_docente/f?p=123:LOGIN_DESKTOP::::::
- **Portal do Colaborador**: https://colaborador.ensinolusofona.pt/mygiaf/Login.xhtml

### Investigação

- **Portal de Investigação**: https://research.ulusofona.pt/
- **ReCiL - Repositório Científico**: https://recil.ensinolusofona.pt/
- **Revistas Científicas**: https://revistas.ulusofona.pt/
- **Unidades de Investigação**: https://investigacao.ulusofona.pt/

### Recursos

- **Biblioteca**: https://biblioteca.ulusofona.pt/
- **Click - Portal de Aprendizagem**: https://www.ulusofona.pt/en/click
- **Documentos**: https://www.ulusofona.pt/documents
- **FAQ - Centro de Ajuda**: https://www.ulusofona.pt/en/faqs
- **Guia de Bem-vindo**: https://bemvindo.ulusofona.pt/
- **Logótipos e Identidade Gráfica**: https://www.ulusofona.pt/documentos?cat=3
- **Objetos Perdidos**: https://www.ulusofona.pt/en/lost-and-found
- **Regulamentos**: https://www.ulusofona.pt/en/documents?cat=1
- **Reshape**: https://secure.ensinolusofona.pt/reshape/
- **Serviços**: https://www.ulusofona.pt/en/services
- **Normas de Dissertações e Teses**: https://www.ulusofona.pt/media/normas-para-elaboracao-e-apresentacao-de-dissertacoes-e-teses.pdf

### Internacional

- **Estudantes Brasileiros**: https://www.ulusofona.pt/en/international-students/brazilian-students
- **Estudantes Internacionais**: https://www.ulusofona.pt/en/international-students
- **FILMEU - Universidade Europeia**: https://www.filmeu.eu/
- **Mobilidade Estudantil**: https://www.ulusofona.pt/en/mobility

### Estudantes

- **Calendário de Defesas de Dissertações**: https://www.ulusofona.pt/en/theses
- **Aplicativo Ensino Lusófona**: https://www.ulusofona.pt/en/services/mobile-app
- **Cartão de Estudante**: https://www.ulusofona.pt/en/news/students-card
- **Estágios**: https://eva.ulusofona.pt/
- **Estudantes**: https://www.ulusofona.pt/en/student
- **Necessidades Educativas Especiais**: https://www.ulusofona.pt/en/gaenee
- **Portal de Emprego**: https://eva.ulusofona.pt/portal-de-emprego-universia/
- **Orientador Estudantil**: https://www.ulusofona.pt/en/student-advisor
- **Bolsas de Estudo**: https://www.ulusofona.pt/en/acao-social-escolar
- **Vantagens e Benefícios**: https://www.ensinolusofona.pt/pt/vantagens

### Comunidade

- **Quartas na Lusófona**: https://www.ulusofona.pt/en/event/as-quartas-na-lusofona-23-24
- **Cinema Fernando Lopes**: https://www.ulusofona.pt/cinema-fernando-lopes
- **Construir Conhecimento**: https://www.ulusofona.pt/en/building-knowledge
- **Jornadas de Abertura**: https://www.ulusofona.pt/en/open-days
- **Escola Sénior**: https://escolasenior.ulusofona.pt/
- **Escola de Verão**: https://escolaverao.ulusofona.pt/
- **Hospital Veterinário - Marcação de Consultas**: https://www.ulusofona.pt/en/news/appointments-veterinary-hospital-
- **Lusófona Talks**: https://www.ulusofona.pt/en/lusofona-talks
- **Lusófona Verde**: https://www.ulusofona.pt/en/green-lusofona

### Media e Eventos

- **Crónicas**: https://www.ulusofona.pt/en/chronicles
- **Lições**: https://www.ulusofona.pt/en/lessons
- **Lusófona nos Media**: https://www.ulusofona.pt/en/lusofona-in-the-media
- **Minha História - Testemunhos**: https://www.ulusofona.pt/en/testimonials
- **Notícias**: https://www.ulusofona.pt/en/news
- **Podcast - Direta Sem Café**: https://www.ulusofona.pt/en/news/direta-sem-cafe-podcast-lusofona

```

'''

class GroqLLM:
    def __init__(self):
        self.llm = Groq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY, temperature=0.1)

    def complete(self, prompt, context=None, system_prompt=SYSTEM_PROMPT):
        messages = [
            ChatMessage(role="system", content=system_prompt),
            ChatMessage(role="user", content=prompt)
        ]
        
        if context is not None:
            messages.append(ChatMessage(role="user", content=f"Context: {context}"))
        
        response = self.llm.chat(messages)
        return response.message.content

    def chat(self, messages):
        response = self.llm.chat(messages)
        return response.message.content

def process_file(file_path, llm):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Step 1: Initial processing
    initial_prompt = f"{content}\n\n{APPEND_PROMPT}"
    initial_response = llm.complete(initial_prompt)

    # Step 2: Revision
    revision_messages = [
        ChatMessage(role="system", content=SYSTEM_PROMPT),
        ChatMessage(role="user", content=initial_prompt),
        ChatMessage(role="assistant", content=initial_response),
        ChatMessage(role="user", content=REVISOR_PROMPT),
    ]
    final_response = llm.chat(revision_messages)

    return final_response #.message.content

def main():
    llm = GroqLLM()
    input_folder = 'output2'
    output_folder = 'output_clean_formatted'

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.md'):  # Assuming the files are markdown
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            print(f"Processing {filename}...")
            processed_content = process_file(input_path, llm)

            # Save the processed content
            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(processed_content)

            print(f"Saved processed content to {output_path}")

if __name__ == "__main__":
    main()

# llm v3

In [35]:
import os
import time
import logging
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage

# Set up logging
logging.basicConfig(filename='processing_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

SYSTEM_PROMPT = '''
Format each document precisely in Markdown, ensuring that absolutely all original content, language, and meaning are preserved in the output. Each message will handle one document at a time, and every single piece of content from the raw input must be included in the formatted output—regardless of whether the document is long or short.

Do not modify or edit any information beyond the necessary formatting adjustments for clarity and consistency. The output must strictly adhere to these rules:

- Avoid generic or redundant phrases describing processes (e.g., stating that something has been reviewed, formatted, or updated).
- Eliminate formal summaries such as 'Here is the result' or 'The document now meets requirements.'
- Focus solely on presenting the content directly, without any explanations of actions taken or process descriptions.
- Ensure the final output is concise, clear, and follows markdown structure, avoiding any filler statements or unnecessary commentary.
- The first line of the output must be exactly the same first line of raw document. Example: "# Response for https://<example.pt>"

The output must be a complete representation of the raw document's content, formatted as required, with no omissions.
'''

#"llama-guard-3-8b"

class GroqLLM:
    def __init__(self):
        def __init__(self):
            self.models = [
                {"name": "llama-3.1-70b-versatile", "context_window": 128000},  # 128k tokens, but max_tokens limited to 8k
                {"name": "llama-3.1-8b-instant", "context_window": 128000},     # 128k tokens, but max_tokens limited to 8k
                {"name": "mixtral-8x7b-32768", "context_window": 32768},
                {"name": "llama3-70b-8192", "context_window": 8192},
                {"name": "llama3-8b-8192", "context_window": 8192},
                {"name": "gemma2-9b-it", "context_window": 8192},
                {"name": "gemma-7b-it", "context_window": 8192},
                {"name": "llama-3.2-90b-vision-preview", "context_window": 8192},  # 128k tokens, but temporarily limited to 8k in preview
                {"name": "llama-3.2-11b-vision-preview", "context_window": 8192},  # 128k tokens, but temporarily limited to 8k in preview
                {"name": "llama3-groq-70b-8192-tool-use-preview", "context_window": 8192},
                {"name": "llama3-groq-8b-8192-tool-use-preview", "context_window": 8192},
                {"name": "llama-3.2-3b-preview", "context_window": 8192},  # 128k tokens, but temporarily limited to 8k in preview
                {"name": "llama-3.2-1b-preview", "context_window": 8192},  # 128k tokens, but temporarily limited to 8k in preview
                {"name": "llama-guard-3-8b", "context_window": 8192},
                {"name": "llava-v1.5-7b-4096-preview", "context_window": 4096},
            ]
            self.current_model_index = 0
            self.llm = self.create_llm()
            self.request_count = 0
            self.last_request_time = 0

    def create_llm(self):
        return Groq(model=self.models[self.current_model_index]["name"], api_key=GROQ_API_KEY, temperature=0.1)
    
    def rotate_model(self):
        self.current_model_index = (self.current_model_index + 1) % len(self.models)
        self.llm = self.create_llm()
        logging.info(f"Switched to model: {self.models[self.current_model_index]}")

    def respect_rate_limit(self):
        current_time = time.time()
        if current_time - self.last_request_time < 60:  # 60 seconds window
            if self.request_count >= 14400 / 1440:  # 14400 requests per day, divided by minutes in a day
                sleep_time = 60 - (current_time - self.last_request_time)
                logging.info(f"Rate limit approached. Sleeping for {sleep_time:.2f} seconds")
                time.sleep(sleep_time)
                self.request_count = 0
                self.last_request_time = time.time()
        else:
            self.request_count = 0
            self.last_request_time = current_time
        self.request_count += 1

    def chat(self, messages, max_retries=3, retry_delay=5):
        original_model_index = self.current_model_index
        for attempt in range(max_retries * len(self.models)):
            try:
                self.respect_rate_limit()
                response = self.llm.chat(messages)
                logging.info(f"Successfully processed using model: {self.models[self.current_model_index]}")
                return response.message.content
            except Exception as e:
                error_message = str(e)
                logging.warning(f"Error with model {self.models[self.current_model_index]}: {error_message}. Attempt {attempt + 1} of {max_retries * len(self.models)}")
                
                if "429" in error_message:
                    # Rate limit error
                    retry_after = int(error_message.split("Please try again in ")[1].split("s.")[0])
                    logging.info(f"Rate limit exceeded. Waiting for {retry_after} seconds before retrying.")
                    time.sleep(retry_after)
                elif "400" in error_message and "maximum context length" in error_message.lower():
                    # Context length error
                    logging.info(f"Context length exceeded for model {self.models[self.current_model_index]}. Rotating to next model.")
                    self.rotate_model()
                elif "404" in error_message and "model does not exist" in error_message.lower():
                    # Model not found error
                    logging.info(f"Model {self.models[self.current_model_index]} not found. Rotating to next model.")
                    self.rotate_model()
                else:
                    # Other errors
                    if attempt % max_retries == max_retries - 1:
                        logging.info(f"Rotating to next model after multiple failures.")
                        self.rotate_model()
                    else:
                        logging.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                
                # If we've tried all models and are back to the original, break the loop
                if self.current_model_index == original_model_index and attempt >= len(self.models):
                    break

        logging.error("Failed to get a response after trying all models and multiple retries.")
        raise Exception("Failed to get a response after trying all models and multiple retries.")

    def get_current_model(self):
        return self.models[self.current_model_index]["name"]

def process_file(file_path, llm):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    messages = [
        ChatMessage(role="system", content=SYSTEM_PROMPT),
        ChatMessage(role="user", content=content)
    ]

    current_model = llm.get_current_model()
    logging.info(f"Processing file {file_path} with model: {current_model}")
    
    response = llm.chat(messages)
    
    logging.info(f"Finished processing {file_path} with model: {current_model}")
    return response

def main():
    llm = GroqLLM()
    input_folder = 'out'
    output_folder = 'output_clean_formatted'

    os.makedirs(output_folder, exist_ok=True)

    files_to_process = [f for f in os.listdir(input_folder) if f.endswith('.md')]
    logging.info(f"Found {len(files_to_process)} .md files in the input folder.")

    for filename in files_to_process:
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        if os.path.exists(output_path):
            logging.info(f"Skipping {filename} as it already exists in the output folder.")
            continue

        logging.info(f"Starting to process {filename}...")
        try:
            processed_content = process_file(input_path, llm)

            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(processed_content)

            logging.info(f"Saved processed content to {output_path}")
        except Exception as e:
            logging.error(f"Error processing {filename}: {str(e)}")
            logging.exception("Exception details:")

    logging.info("Finished processing all files.")

if __name__ == "__main__":
    main()

In [36]:
class GroqLLM:
    def __init__(self):
        self.models = [
            {"name": "llama-3.1-70b-versatile", "context_window": 8192},
            {"name": "llama-3.1-8b-instant", "context_window": 8192},
            {"name": "mixtral-8x7b-32768", "context_window": 32768},
            {"name": "llama3-70b-8192", "context_window": 8192},
            {"name": "llama3-8b-8192", "context_window": 8192},
            {"name": "gemma2-9b-it", "context_window": 8192},
            {"name": "gemma-7b-it", "context_window": 8192},
            {"name": "llama-3.2-90b-vision-preview", "context_window": 8192},
            {"name": "llama-3.2-11b-vision-preview", "context_window": 8192},
            {"name": "llama3-groq-70b-8192-tool-use-preview", "context_window": 8192},
            {"name": "llama3-groq-8b-8192-tool-use-preview", "context_window": 8192},
            {"name": "llama-3.2-3b-preview", "context_window": 8192},
            {"name": "llama-3.2-1b-preview", "context_window": 8192},
            {"name": "llama-guard-3-8b", "context_window": 8192},
            {"name": "llava-v1.5-7b-4096-preview", "context_window": 4096},
        ]
        self.current_model_index = 0
        self.llm = None
        self.request_count = 0
        self.last_request_time = 0
        self.initialize_llm()

    def initialize_llm(self):
        self.llm = Groq(model=self.models[self.current_model_index]["name"], api_key=GROQ_API_KEY, temperature=0.1)

    def rotate_model(self):
        self.current_model_index = (self.current_model_index + 1) % len(self.models)
        self.initialize_llm()
        logging.info(f"Switched to model: {self.get_current_model()}")

    def respect_rate_limit(self):
        current_time = time.time()
        if current_time - self.last_request_time < 60:  # 60 seconds window
            if self.request_count >= 14400 / 1440:  # 14400 requests per day, divided by minutes in a day
                sleep_time = 60 - (current_time - self.last_request_time)
                logging.info(f"Rate limit approached. Sleeping for {sleep_time:.2f} seconds")
                time.sleep(sleep_time)
                self.request_count = 0
                self.last_request_time = time.time()
        else:
            self.request_count = 0
            self.last_request_time = current_time
        self.request_count += 1

    def chat(self, messages, max_retries=3, retry_delay=5):
        original_model_index = self.current_model_index
        for attempt in range(max_retries * len(self.models)):
            try:
                self.respect_rate_limit()
                response = self.llm.chat(messages)
                logging.info(f"Successfully processed using model: {self.get_current_model()}")
                return response.message.content
            except Exception as e:
                error_message = str(e)
                logging.warning(f"Error with model {self.get_current_model()}: {error_message}. Attempt {attempt + 1} of {max_retries * len(self.models)}")
                
                if "429" in error_message:
                    # Rate limit error
                    retry_after = int(error_message.split("Please try again in ")[1].split("s.")[0])
                    logging.info(f"Rate limit exceeded. Waiting for {retry_after} seconds before retrying.")
                    time.sleep(retry_after)
                elif "400" in error_message and "maximum context length" in error_message.lower():
                    # Context length error
                    logging.info(f"Context length exceeded for model {self.get_current_model()}. Rotating to next model.")
                    self.rotate_model()
                elif "404" in error_message and "model does not exist" in error_message.lower():
                    # Model not found error
                    logging.info(f"Model {self.get_current_model()} not found. Rotating to next model.")
                    self.rotate_model()
                else:
                    # Other errors
                    if attempt % max_retries == max_retries - 1:
                        logging.info(f"Rotating to next model after multiple failures.")
                        self.rotate_model()
                    else:
                        logging.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                
                # If we've tried all models and are back to the original, break the loop
                if self.current_model_index == original_model_index and attempt >= len(self.models):
                    break

        logging.error("Failed to get a response after trying all models and multiple retries.")
        raise Exception("Failed to get a response after trying all models and multiple retries.")

    def get_current_model(self):
        return self.models[self.current_model_index]["name"]
    



def process_file(file_path, llm):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    messages = [
        ChatMessage(role="system", content=SYSTEM_PROMPT),
        ChatMessage(role="user", content=content)
    ]

    current_model = llm.get_current_model()
    logging.info(f"Processing file {file_path} with model: {current_model}")
    
    response = llm.chat(messages)
    
    logging.info(f"Finished processing {file_path} with model: {current_model}")
    return response


def main():
    try:
        llm = GroqLLM()
        input_folder = 'out'
        output_folder = 'output_clean_formatted'

        os.makedirs(output_folder, exist_ok=True)

        files_to_process = [f for f in os.listdir(input_folder) if f.endswith('.md')]
        logging.info(f"Found {len(files_to_process)} .md files in the input folder.")

        for filename in files_to_process:
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            if os.path.exists(output_path):
                logging.info(f"Skipping {filename} as it already exists in the output folder.")
                continue

            logging.info(f"Starting to process {filename}...")
            try:
                processed_content = process_file(input_path, llm)

                with open(output_path, 'w', encoding='utf-8') as file:
                    file.write(processed_content)

                logging.info(f"Saved processed content to {output_path}")
            except Exception as e:
                logging.error(f"Error processing {filename}: {str(e)}")
                logging.exception("Exception details:")

        logging.info("Finished processing all files.")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {str(e)}")
        logging.exception("Exception details:")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

# optimize

In [38]:
import time
import logging
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage

class GroqLLM:
    def __init__(self):
        self.models = [
            {"name": "llama-3.1-70b-versatile", "context_window": 8192},
            {"name": "llama-3.1-8b-instant", "context_window": 8192},
            {"name": "mixtral-8x7b-32768", "context_window": 32768},
            {"name": "llama3-70b-8192", "context_window": 8192},
            {"name": "llama3-8b-8192", "context_window": 8192},
            {"name": "gemma2-9b-it", "context_window": 8192},
            {"name": "gemma-7b-it", "context_window": 8192},
            {"name": "llama-3.2-90b-vision-preview", "context_window": 8192},
            {"name": "llama-3.2-11b-vision-preview", "context_window": 8192},
            {"name": "llama3-groq-70b-8192-tool-use-preview", "context_window": 8192},
            {"name": "llama3-groq-8b-8192-tool-use-preview", "context_window": 8192},
            {"name": "llama-3.2-3b-preview", "context_window": 8192},
            {"name": "llama-3.2-1b-preview", "context_window": 8192},
            {"name": "llama-guard-3-8b", "context_window": 8192},
            {"name": "llava-v1.5-7b-4096-preview", "context_window": 4096},
        ]
        self.current_model_index = 0
        self.last_request_time = 0
        self.requests_in_window = 0
        self.rate_limit = 500000  # tokens per minute
        self.window_size = 60  # 1 minute window

    def get_current_model(self):
        return self.models[self.current_model_index]

    def rotate_model(self):
        self.current_model_index = (self.current_model_index + 1) % len(self.models)
        logging.info(f"Rotating to model: {self.get_current_model()['name']}")

    def wait_for_rate_limit(self, tokens_requested):
        current_time = time.time()
        time_since_last_request = current_time - self.last_request_time

        if time_since_last_request >= self.window_size:
            self.requests_in_window = 0
        else:
            self.requests_in_window -= max(0, (self.requests_in_window * time_since_last_request / self.window_size))

        if self.requests_in_window + tokens_requested > self.rate_limit:
            wait_time = (self.requests_in_window + tokens_requested - self.rate_limit) / (self.rate_limit / self.window_size)
            logging.info(f"Rate limit approached. Waiting for {wait_time:.2f} seconds")
            time.sleep(wait_time)

        self.requests_in_window += tokens_requested
        self.last_request_time = time.time()

    def chat(self, messages, max_retries=3):
        for attempt in range(max_retries * len(self.models)):
            current_model = self.get_current_model()
            llm = Groq(model=current_model["name"])

            try:
                # Estimate token count (this is a rough estimate, you may want to use a proper tokenizer)
                tokens_requested = sum(len(msg.content.split()) for msg in messages) * 1.3

                self.wait_for_rate_limit(tokens_requested)

                response = llm.chat(messages)
                return response.message.content

            except Exception as e:
                error_message = str(e)
                logging.warning(f"Error with model {current_model['name']}: {error_message}. Attempt {attempt + 1} of {max_retries * len(self.models)}")

                if "429" in error_message:
                    retry_after = 60  # Default to 60 seconds if we can't parse the time
                    try:
                        retry_after = int(float(error_message.split("Please try again in ")[1].split("s")[0]))
                    except:
                        pass
                    logging.info(f"Rate limit exceeded. Waiting for {retry_after} seconds before retrying.")
                    time.sleep(retry_after)
                    self.rotate_model()
                elif attempt % max_retries == max_retries - 1:
                    self.rotate_model()
                else:
                    time.sleep(5)  # Wait for 5 seconds before retrying with the same model

        logging.error("Failed to get a response after trying all models and multiple retries.")
        return None

def process_file(file_path, llm):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        messages = [
            ChatMessage(role="system", content="You are a helpful assistant that cleans and formats text."),
            ChatMessage(role="user", content=f"Please clean and format the following text, removing any HTML tags, fixing formatting issues, and ensuring it's well-structured:\n\n{content}")
        ]

        processed_content = llm.chat(messages)
        return processed_content
    except Exception as e:
        logging.error(f"Failed to process {file_path}: {str(e)}")
        return None
    
def main():
    llm = GroqLLM()
    input_folder = 'out'
    output_folder = 'output_clean_formatted'
    failed_folder = 'failed_processing'

    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(failed_folder, exist_ok=True)

    files_to_process = [f for f in os.listdir(input_folder) if f.endswith('.md')]
    logging.info(f"Found {len(files_to_process)} .md files in the input folder.")

    for filename in files_to_process:
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        if os.path.exists(output_path):
            logging.info(f"Skipping {filename} as it already exists in the output folder.")
            continue

        logging.info(f"Starting to process {filename}...")
        processed_content = process_file(input_path, llm)

        if processed_content:
            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(processed_content)
            logging.info(f"Saved processed content to {output_path}")
        else:
            failed_path = os.path.join(failed_folder, filename)
            os.rename(input_path, failed_path)
            logging.warning(f"Moved {filename} to {failed_folder} due to processing failure.")

    logging.info("Finished processing all files.")

if __name__ == "__main__":
    main()



# Script clean output_clean_formatted folder of files create from a llama-2 call gone wrong

In [30]:
import os
import re

def delete_matching_files(log_file_path, output_folder):
    """
    Reads through a log file, finds matching files and deletes them from the output folder.

    Args:
    - log_file_path (str): Path to the log file.
    - output_folder (str): Path to the output folder.

    Returns:
    - None
    """
    try:
        # Read the log file
        with open(log_file_path, 'r') as file:
            for line in file:
                # Use regular expression to find the file name
                match = re.search(r'Finished processing out/(.+?).md with model: llama-2-70b-chat', line)
                if match:
                    file_name = match.group(1)
                    file_path = os.path.join(output_folder, file_name + '.md')
                    if os.path.exists(file_path):
                        # Delete the file
                        os.remove(file_path)
                        print(f"Deleted file: {file_path}")
                    else:
                        print(f"File not found: {file_path}")
    
    except FileNotFoundError:
        print(f"Log file not found: {log_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Example usage
log_file_path = 'processing_log.txt'
output_folder = 'output_clean_formatted'
delete_matching_files(log_file_path, output_folder)

Deleted file: output_clean_formatted/en_teachers_julio-do-carmo-barata-3048.md
Deleted file: output_clean_formatted/docentes_angelina-maria-da-costa-santos-2434.md
Deleted file: output_clean_formatted/lisboa_licenciaturas_comunicacao-e-jornalismo_ULHT449-8403.md
Deleted file: output_clean_formatted/porto_licenciaturas_ciencias-da-comunicacao_ULP451-26060.md
Deleted file: output_clean_formatted/docentes_maria-de-lurdes-coentro-vargas-6105.md
Deleted file: output_clean_formatted/lisboa_licenciaturas_cinema-e-artes-dos-media_ULHT160-24626.md
Deleted file: output_clean_formatted/en_teachers_jose-gregorio-viegas-bras-543.md
Deleted file: output_clean_formatted/docentes_catarina-de-moura-coelho-6840.md
Deleted file: output_clean_formatted/docentes_flavio-jose-serrano-roques-2101.md
Deleted file: output_clean_formatted/en_news_sustainability-movement-.md
Deleted file: output_clean_formatted/lisboa_licenciaturas_design-e-producao-de-moda_ULP6805-25318.md
Deleted file: output_clean_formatted/en

In [14]:
%pip uninstall python-dotenv -y


Found existing installation: python-dotenv 1.0.1
Uninstalling python-dotenv-1.0.1:
  Successfully uninstalled python-dotenv-1.0.1
Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.
